In [2]:
import cv2
import numpy as np

In [24]:
HORI_RES = 180
ASPECT = 2.5 # Height of Character / Width of Character in terminal

def resize(img, hori_res, character_aspect_ratio):
    new_size = (hori_res, int(hori_res / character_aspect_ratio * img.shape[0] // img.shape[1]))
    return cv2.resize(img, new_size)    

def black_white(img):
    return (~np.round(img / 255).astype(bool)).astype(int)



In [14]:
def to_ascii(img):
    with open("shitty_ascii.txt", "w") as f:
        write = "\n".join(["".join([" " if x == 0 else '#' for x in row]) for row in img])
        f.write(write)

def to_text_python_list(img):
    with open("python_list.txt", "w") as f:
        f.write(str(img.tolist()))

def to_text_lisp_list(img):
    with open("lisp_list.txt", "w") as f:
        f.write("'" + str(img.tolist()).replace(", ", " ").replace("[", "(").replace("]", ")"))


In [25]:
img = cv2.imread(f"bad_apple/image_sequence/bad_apple_{291}.png", cv2.IMREAD_GRAYSCALE)
img = resize(img, HORI_RES, ASPECT)
img = black_white(img)

to_text_lisp_list(img)

In [12]:
def create_wrapped_text(frame, text):
    word_count = 0
    full_wrapped_text = ""

    for i in range(len(frame)):
        pixel_count = 0
        wrapped_line = ""

        row = frame[i]

        while pixel_count < len(row):
            word = text[word_count] + " "
        
            word_len = len(word)
            look_ahead = row[pixel_count:pixel_count+word_len]
            look_ahead.extend([0] * (word_len - len(look_ahead))) # pad to avoid possible bug where if we are at the end of the row e.g. [1, 0, 0] but the word is longer e.g. "lisp", then the arrays are no longer the same length and we can't xor them

            word_reduced = [0 if x == " " else 1 for x in word]

            if sum(~np.logical_xor(word_reduced, look_ahead)) / len(look_ahead) > .6: 
                wrapped_line += word
                
                word_count += 1
                pixel_count += word_len
            else:
                wrapped_line += " "
                pixel_count += 1

        full_wrapped_text += wrapped_line + "\n"

    return full_wrapped_text


In [13]:
NUM_FRAMES = 520

lorem_ipsum_text = open("lorem_ipsum.txt").read().split()

for i in range(1, NUM_FRAMES):
    str_i = str(i)

    while len(str_i) < 3:
        str_i = "0" + str_i

    img = cv2.imread(f"bad_apple/image_sequence/bad_apple_{str_i}.png", cv2.IMREAD_GRAYSCALE)
    img = resize(img, HORI_RES, ASPECT)
    img = black_white(img)
    img = img.tolist()

    wrap_text = create_wrapped_text(img, lorem_ipsum_text)

    with open(f"output/wrapped_output_{str(i)}.txt", "w") as f:
        f.write(wrap_text)
    
    